In [1]:
cd models/research/object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [3]:
import numpy as np
import math
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.layers import *

from keras.applications import MobileNetV2
from keras.applications import InceptionResNetV2

from keras.models import Model
from keras.models import model_from_json


In [4]:
target_size = [480, 480]
target_w = 480 # target sizes of image in model input
target_h = 480 #target sizes of image in model input

grid_size = [15, 15]
grid_y_axis = 15  # each image is to be segmented to 13 x 13 grid
grid_x_axis = 15  # # each image is to be segmented to 13 x 13 grid

grid_w = target_w / grid_x_axis  # grid cell width
grid_h = target_h / grid_y_axis  # grid cell height

channels = 3
num_anchors = 3
class_num = 5 # vendor, invoice, inv_date, po, buyer
info = 5 + class_num    # pc, x, y, h, w, and class probabilities

categories = ['vendor', 'invoice', 'inv_date', 'po', 'buyer'] # details of classes

In [18]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/8-6-new-scan'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_new_image = ['/home/scar3crow/Downloads/8-6-new-scan/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_new_image.sort() # Sorting the list

num_images = len(inv_new_image)

print('Number of images = ', num_images)
inv_new_image[24]

Number of images =  36


'/home/scar3crow/Downloads/8-6-new-scan/50a.jpg'

In [19]:
# Check sizes of exiting images & Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df_new = pd.DataFrame()

for i in range(len(inv_new_image)):
    image = cv2.imread(inv_new_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_new_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df_new.insert(loc=0, column='image_serial', value=image_num)
df_new.insert(loc=1, column='rows', value=row_values)
df_new.insert(loc=2, column='columns', value=col_values)

df_new.head(3)

,image_serial,rows,columns
0,/home/scar3crow/Downloads/8-6-new-scan/101a.jpg,160,416
1,/home/scar3crow/Downloads/8-6-new-scan/102a.jpg,406,870
2,/home/scar3crow/Downloads/8-6-new-scan/103a.jpg,260,416


In [5]:
def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = Conv2D(conv['filter'], 
                   conv['kernel'], 
                   strides=conv['stride'], 
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']), 
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)

    return add([skip_connection, x]) if skip else x


In [15]:
def make_yolov3_model():
    input_image = Input(shape=(480, 480, 3))

    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])

    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])

    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])

    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])

    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
        
    skip_36 = x
        
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])

    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
        
    skip_61 = x
        
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
    
    # Layer 62 => 65
##    x = _conv_block(x, [{'filter':  256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
##                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
##                        {'filter':  256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])


    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
    
     # Layer 66 => 74
##    for i in range(3):
##        x = _conv_block(x, [{'filter':  128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
##                            {'filter':  256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])

    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
    
    # Layer 75 => 79
##    x = _conv_block(x, [{'filter':  128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
##                        {'filter':  256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
##                        {'filter':  128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
##                        {'filter':  256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
##                        {'filter':  128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
    
    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])

    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)

    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 101},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 102},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 103}])

    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter':  30, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

#===============================================================================================================
#    # Layer 95 => 98
#    x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
#    x = UpSampling2D(2)(x)
#    x = concatenate([x, skip_36])

#    # Layer 99 => 106
#    yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
#                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
#                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
#                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
#                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
#                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
#                               {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)

#    model = Model(input_image, [yolo_82, yolo_94, yolo_106])

    final = Reshape((grid_y_axis,grid_x_axis,num_anchors,info))(yolo_94)
    model = Model(input_image, final)
    return model


In [16]:
input_size = (target_w, target_h, 3)
my_yolo_invoice_model = make_yolov3_model()


my_model_1 = make_yolov3_model()
my_model_2 = make_yolov3_model()
my_model_3 = make_yolov3_model()

print(my_model_1.summary())


Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 480, 480, 32) 864         input_15[0][0]                   
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 480, 480, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, 480, 480, 32) 0           bnorm_0[0][0]                    
___________________________________________________________________________________________

In [20]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_new_data = pd.read_csv('/home/scar3crow/Downloads/via_new_data.csv')
num_obj = r_new_data['region_count'][0] # number of objects in each photo
r_new_data.drop(r_new_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_new_data.sort_values(by=['#filename'], ascending=True) # Sorting based on image-id
num_images = r_new_data["#filename"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_new_data[58:61]

Number of classes =  5
Number of unique images =  36


,#filename,region_shape_attributes,region_attributes
58,63a.jpg,"{""name"":""rect"",""x"":211,""y"":64,""width"":76,""heig...","{""text"":""po""}"
59,63a.jpg,"{""name"":""rect"",""x"":2,""y"":68,""width"":165,""heigh...","{""text"":""buyer""}"
60,101a.jpg,"{""name"":""rect"",""x"":6,""y"":23,""width"":119,""heigh...","{""text"":""vendor""}"


In [21]:
# Making a dataframe for Image_id, x, y, width, height, class, image_width and image_height

x = []
y = []
width = []
height = []
obj_class = []
i_width = []
i_height = []
img_path = []
img_index = []

for i in range(len(r_new_data)):
    
    r_size = r_new_data.values[i, 1][1:(len(r_new_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_new_data.values[i, 2][1:(len(r_new_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
    foto_id = r_new_data['#filename'][i]
    i_path = '/home/scar3crow/Downloads/8-6-new-scan/' + foto_id
    foto_index = int(df_new[df_new['image_serial'] == i_path].index[0])
    foto_width = df_new['columns'][foto_index]
    foto_height = df_new['rows'][foto_index]
    i_width.append(foto_width)
    i_height.append(foto_height)
    img_path.append(i_path)
    img_index.append(foto_index)
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)
i_width_values = pd.Series(i_width)
i_height_values = pd.Series(i_height)
img_path_values = pd.Series(img_path)
img_index_values = pd.Series(img_index)

r_new_data.insert(loc=1, column='img_idx', value=img_index_values)
r_new_data.insert(loc=2, column='i_path', value=img_path_values)
r_new_data.insert(loc=3, column='x', value=x_values)
r_new_data.insert(loc=4, column='y', value=y_values)
r_new_data.insert(loc=5, column='width', value=width_values)
r_new_data.insert(loc=6, column='height', value=height_values)
r_new_data.insert(loc=7, column='obj_class', value=class_values)
r_new_data.insert(loc=8, column='img_wd', value=i_width_values)
r_new_data.insert(loc=9, column='img_ht', value=i_height_values)

r_new_data.drop(r_new_data.columns[[10, 11]], axis=1, inplace=True) # reduce unnecessary columns

r_new_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_new_data[3:6]


,img_id,img_idx,i_path,x,y,width,height,obj_class,img_wd,img_ht
3,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,221,59,103,24,po,416,209
4,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,5,57,206,56,buyer,416,209
5,51a.jpg,25,/home/scar3crow/Downloads/8-6-new-scan/51a.jpg,5,0,120,56,vendor,416,194


In [22]:
print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

Number of unique images =  36
Number of classes in diff. categories =  buyer      38
date       36
vendor     36
invoice    36
po         33
order       1
Name: obj_class, dtype: int64


In [23]:
# We have to correct above :

# To find smallest width & height boxes in 'buyer' which should be 'po'
gb = r_new_data.groupby('obj_class')    
[gb.get_group('buyer') for x in gb.groups]

[       img_id  img_idx                                           i_path    x  \
 4     50a.jpg       24   /home/scar3crow/Downloads/8-6-new-scan/50a.jpg    5   
 9     51a.jpg       25   /home/scar3crow/Downloads/8-6-new-scan/51a.jpg    4   
 14    52a.jpg       26   /home/scar3crow/Downloads/8-6-new-scan/52a.jpg    1   
 19    53a.jpg       27   /home/scar3crow/Downloads/8-6-new-scan/53a.jpg    0   
 24    54a.jpg       28   /home/scar3crow/Downloads/8-6-new-scan/54a.jpg   31   
 29    55a.jpg       29   /home/scar3crow/Downloads/8-6-new-scan/55a.jpg    1   
 34    56a.jpg       30   /home/scar3crow/Downloads/8-6-new-scan/56a.jpg    1   
 39    59a.jpg       31   /home/scar3crow/Downloads/8-6-new-scan/59a.jpg    3   
 44    60a.jpg       32   /home/scar3crow/Downloads/8-6-new-scan/60a.jpg    0   
 49    61a.jpg       33   /home/scar3crow/Downloads/8-6-new-scan/61a.jpg    1   
 54    62a.jpg       34   /home/scar3crow/Downloads/8-6-new-scan/62a.jpg    4   
 59    63a.jpg       35   /h

In [24]:
# Correcting above wrong spelling & converting buyer to po of object classes and rechecking

id_1 = r_new_data.index[r_new_data['obj_class'] == 'order'] # Finding the index
id_2 = r_new_data.index[r_new_data['obj_class'] == 'date'] # to change 'date' to 'inv_date' to be consistent with old data

r_new_data.at[id_1, 'obj_class'] = 'po' # writing the correct spelling 
r_new_data.at[88, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[163, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[id_2, 'obj_class'] = 'inv_date' # # 'date' to 'inv_date'

print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_new_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

# r_new_data.drop(r_new_data.columns[[0]], axis=1, inplace=True) # reduce unnecessary columns

Number of unique images =  36
Number of unique classes =  5
Number of classes in diff. categories =  buyer       36
inv_date    36
po          36
vendor      36
invoice     36
Name: obj_class, dtype: int64


In [25]:
## For each image, we have to find : (a) line_index = integer, (b) img_path = string, (c) boxes = shape [N, 4], 
## N is the ground truth count, elements in the second dimension are [x_min, y_min, x_max, y_max] (d) labels = shape
## [N]. class index. (e) img_width = int.  =f) img_height = int

def single_image_info(lines):
    
    ## lines will be a dataframe like, for i in range(num_images), lines = r_new_data[i*5:(i+1)*5]
    
    line_idx = lines.iat[0, 1]
    pic_path = lines.iat[0, 2]
    img_width = lines.iat[0, 8]
    img_height = lines.iat[0, 9]
    
    boxes = []
    labels = []
    for i in range(len(lines)):
        label, x_min, y_min, x_max, y_max = int(i), float(lines.iat[i,3]), float(lines.iat[i,4]), float(lines.iat[i,3]+lines.iat[i,5]), float(lines.iat[i,4]+lines.iat[i,6])
        boxes.append([x_min, y_min, x_max, y_max])
        labels.append(label)
        
    boxes = np.asarray(boxes, np.float32)
    labels = np.asarray(labels, np.int64)
    
    return line_idx, pic_path, boxes, labels, img_width, img_height  ## boxes are in format xmin, ymin, xmax, ymax


In [26]:
## Creating the complete data set :

all_image_line = []
for i in range(num_images):
    image_line = []
    limit_lower = i*5
    limit_upper = limit_lower+5
    lines = r_new_data[limit_lower:limit_upper]
    line_idx, pic_path, boxes, labels, img_width, img_height = single_image_info(lines)
    image_line.append(line_idx)
    image_line.append(pic_path)
    image_line.append(boxes)
    image_line.append(labels)
    image_line.append(img_width)
    image_line.append(img_height)
    all_image_line.append(image_line)
    
print(len(all_image_line))
print(all_image_line[1])   ##  boxes are in format xmin, ymin, xmax, ymax

36
[25, '/home/scar3crow/Downloads/8-6-new-scan/51a.jpg', array([[  5.,   0., 125.,  56.],
       [239.,   1., 279.,  20.],
       [328.,   1., 382.,  21.],
       [238.,  51., 302.,  74.],
       [  4.,  53., 156., 117.]], dtype=float32), array([0, 1, 2, 3, 4]), 416, 194]


In [27]:
# Train and Test split

data_train, data_val = train_test_split(all_image_line, train_size = 0.8 , shuffle = True)

num_all_bbox = len(all_image_line) * len(all_image_line[0][2])
num_bb_train = len(data_train) * len(data_train[0][2])
num_bb_val = len(data_val) * len(data_val[0][2])
print(num_all_bbox, num_bb_train, num_bb_val)

180 140 40


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [28]:
## calculating anchors from true boundary boxes :

def iou_kmeans(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    :param box: tuple or array, shifted to the origin (i. e. width and height)
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou = intersection / (box_area + cluster_area - intersection)

    return iou

def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param k: number of clusters
    :param dist: distance function
    :return: numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]


    while True:
        for row in range(rows):
            distances[row] = 1 - iou_kmeans(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

In [31]:
## Finding out anchors :
## Firstly, converting true boundary box width, height to width & height with respect to target image :
## finaly find anchors. Anchors here are in absolute size w.r.t. target image but not as % of target image or 
## as multiple of unit grids.

# num_all_bb = len(r_new_data) # if no. of bboxes varies for images, this formula should be used 

anchors_wrt_target = np.zeros((3,2))

num_all_bb = len(all_image_line) * len(all_image_line[0][2])  ## from all image line data

b_box_wrt_target = np.zeros((num_all_bb,2))

for i in range(num_all_bb):
    
    image_w = r_new_data['img_wd'][i]
    image_h = r_new_data['img_ht'][i]

    x_ratio = target_w / image_w 
    y_ratio = target_h / image_h
    
    anchor_w = r_new_data['width'][i] * x_ratio
    anchor_h = r_new_data['height'][i] * y_ratio
    b_box_wrt_target[i, 0] = anchor_w
    b_box_wrt_target[i, 1] = anchor_h
    
anchors_wrt_target = kmeans(b_box_wrt_target, num_anchors)

print(anchors_wrt_target.shape)
print(anchors_wrt_target)     ## anchors wrt target image in abs. value and in format width, height


(3, 2)
[[ 68.07692308  65.30612245]
 [129.23076923  37.92592593]
 [189.80769231 160.36697248]]


In [71]:
## Pre-processing the original data to get y_true :

def process_box(ori_boxes, ori_img_width, ori_img_height, labels, target_size, class_num, anchors):
    '''
    Generate the y_true label, i.e. the ground truth feature_map.
    params:
        boxes: [N, 5] shape, float32 dtype. `x_min, y_min, x_max, y_mix, mixup_weight`.
        labels: [N] shape, int64 dtype.
        class_num: int64 num.
        anchors: [3,2] shape, float32 dtype.
    '''
    
    img_width = ori_img_width
    img_height = ori_img_height
    boxes = ori_boxes           ## boxes in format xmin, ymin, xmax, ymax
    
    x_ratio = target_size[1] / img_width
    y_ratio = target_size[0] / img_height
    
    boxes_wrt_target = np.zeros((5,4))
    box_centers_target = np.zeros((5,2))

    boxes_wrt_target[:,0] = boxes[:,0] * x_ratio  # xmin absolute value wrt target image
    boxes_wrt_target[:,1] = boxes[:,1] * y_ratio  # ymin absolute value wrt target image
    boxes_wrt_target[:,2] = boxes[:,2] * x_ratio  # xmax absolute value wrt target image
    boxes_wrt_target[:,3] = boxes[:,3] * y_ratio  # ymax absolute value wrt target image
    
    # In above, boxes_wrt_target shape is (5, 4), now this will be taken to (5. 5) by adding 1 at end
#    boxes_wrt_target = np.concatenate((boxes_wrt_target, np.full(shape=(boxes_wrt_target.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)
    box_centers_target = (boxes_wrt_target[:, 0:2] + boxes_wrt_target[:, 2:4]) / 2  ## centers wrt target, abs values
    
    box_sizes = boxes[:, 2:4] - boxes[:, 0:2]  #xmax-xmin = width and ymax-ymin = height wrt original image
    box_sizes[:,0] = box_sizes[:,0] * x_ratio  # width w.r.t target image in absolute value
    box_sizes[:,1] = box_sizes[:,1] * y_ratio  # width w.r.t target image in absolute value
    
#    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 6 + class_num), np.float32)
    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 5 + class_num), np.float32)

#    y_true = [y_true_13]
    
    box_sizes = np.expand_dims(box_sizes, 1)
    mins = np.maximum(- box_sizes / 2, - anchors / 2)
    maxs = np.minimum(box_sizes / 2, anchors / 2)
    whs = maxs - mins

    iou = (whs[:, :, 0] * whs[:, :, 1]) / (
                box_sizes[:, :, 0] * box_sizes[:, :, 1] + anchors[:, 0] * anchors[:, 1] - whs[:, :, 0] * whs[:, :,
                                                                                                         1] + 1e-10)
    best_match_idx = np.argmax(iou, axis=1)
    
    print(best_match_idx)

    anchor_mask = np.zeros((target_size[1] // 32, target_size[0] // 32, 3))

    grid_stride = 32  ## = targetsize / no. of grid cells
    
    for i, idx in enumerate(best_match_idx):

        x = int(np.floor(box_centers_target[i, 0] / grid_stride))
        y = int(np.floor(box_centers_target[i, 1] / grid_stride))
        k = int(idx)
        c = int(labels[i])

        print(x, y, k, c)

# Very Imp : Now preparing y_true: all values x_center, y_cemter, width & height are being taken to % of target image
        
        y_true_13[y, x, k, :2] = box_centers_target[i] / target_size[0] #since target_size[0] = target_size[1]
        y_true_13[y, x, k, 2:4] = box_sizes[i] / target_size[0]
        y_true_13[y, x, k, 4] = 1.
        y_true_13[y, x, k, 5 + c] = 1.
#        y_true[0][y, x, k, -1] = boxes_wrt_target[i, -1]
        anchor_mask[y, x, k] = 1

    return y_true_13, anchor_mask  ## all data are w.r.to target image in % of target image and NOT w,r,t, grid cells


In [33]:
## Single image-wise image/boundary box preprocessing:

def parse_data(line, class_num, target_size, anchors):   ## (mode, letterbox_resize):
    '''
    param:
        line: a line from the training/test txt file
        class_num: totol class nums.
        target_size: the size of image to be resized to. [width, height] format.
        anchors: anchors.
        mode: 'train' or 'val'. When set to 'train', data_augmentation will be applied.
        letterbox_resize: whether to use the letterbox resize, i.e., keep the original aspect ratio in the resized image.
    '''
    
    img_idx, pic_path, boxes, labels,img_width, img_height = line  # boxes in format xmin, ymin, xmax, ymax
    img = cv2.imread(pic_path)
    img_resized = cv2.resize(img,(target_size[0], target_size[1]))
    
    # expand the 2nd dimension, mix up weight default to 1.
    boxes = np.concatenate((boxes, np.full(shape=(boxes.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)

    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32)

    # the input of yolo_v3 should be in range 0~1, lets change to -0.5 to +0.5
    img_resized = (img_resized - 127.5)/ 255.

    y_true_13, anchor_mask = process_box(boxes, img_width, img_height, labels, target_size, class_num, anchors)

    return img_idx, img_resized, y_true_13, anchor_mask


In [72]:
line = all_image_line[1]
anchors = anchors_wrt_target
    
img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)


[2 0 0 0 2]
2 2 2 0
9 0 0 1
12 0 0 2
9 4 0 3
2 6 2 4


In [34]:
## Making the data ready for entering into network :

anchors = anchors_wrt_target
image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_train)):

    line = data_train[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    
    
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
    
train_image_index = image_index
X_train = np.array(image_resized).astype(np.float32)
Y_train = np.array(image_y_true).astype(np.float32)
train_anchor_mask = np.array(image_anchor_mask).astype(np.float32)

image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_val)):
    line = data_val[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
val_image_index = image_index
X_val = np.array(image_resized).astype(np.float32)
Y_val = np.array(image_y_true).astype(np.float32)
val_anchor_mask = np.array(image_anchor_mask).astype(np.float32)

image_index = []
image_resized = []
image_y_true = []


4 3 2 0
9 1 0 1
12 1 0 2
9 4 0 3
3 10 2 4
2 2 2 0
9 1 0 1
12 0 0 2
9 5 0 3
3 6 2 4
5 2 2 0
2 5 1 1
2 4 1 2
2 7 1 3
10 12 2 4
2 2 2 0
8 1 0 1
12 1 0 2
8 6 0 3
3 7 2 4
7 5 2 0
10 10 1 1
10 11 1 2
11 13 1 3
4 12 2 4
3 2 2 0
9 1 0 1
12 1 0 2
9 5 1 3
3 6 2 4
2 3 1 0
9 5 1 1
9 7 1 2
9 12 1 3
3 9 2 4
2 3 2 0
9 1 0 1
12 1 0 2
10 7 0 3
3 10 2 4
4 2 2 0
9 1 0 1
12 0 0 2
10 4 0 3
3 7 2 4
2 2 2 0
9 1 0 1
12 2 0 2
10 7 0 3
2 7 2 4
2 2 2 0
9 1 0 1
12 1 0 2
9 5 0 3
3 7 2 4
2 3 2 0
9 1 0 1
12 1 0 2
9 8 0 3
3 10 2 4
11 4 2 0
9 9 1 1
13 9 1 2
9 12 1 3
3 12 2 4
5 1 2 0
2 4 1 1
2 4 1 2
2 6 1 3
10 12 2 4
11 3 2 0
9 9 0 1
13 9 1 2
10 12 1 3
4 11 2 4
2 2 2 0
9 2 0 1
12 2 0 2
9 8 0 3
2 9 2 4
2 4 2 0
9 3 0 1
12 2 0 2
10 7 0 3
2 8 2 4
3 2 2 0
9 1 0 1
12 1 0 2
9 5 0 3
2 6 2 4
3 6 2 0
9 5 0 1
12 5 0 2
9 8 1 3
3 9 2 4
2 2 2 0
8 1 0 1
12 1 0 2
9 6 0 3
3 7 2 4
3 2 2 0
9 1 0 1
12 1 0 2
9 5 0 3
3 6 2 4
2 2 2 0
9 0 0 1
12 0 0 2
9 3 0 3
3 6 2 4
2 2 2 0
9 0 0 1
12 0 0 2
9 4 0 3
2 6 2 4
4 3 2 0
9 1 0 1
12 1 0 2
9 5 0 3
3 

In [35]:
print(len(X_train), len(Y_train), len(train_image_index), len(X_val), len(Y_val), len(val_image_index))
print(train_image_index)
print(X_train.shape, Y_train.shape)
print(train_image_index[0], X_train[0].shape, Y_train[0].shape)
print(train_anchor_mask.shape, 'and', train_anchor_mask[0].shape)

28 28 28 8 8 8
[18, 16, 7, 35, 9, 24, 12, 30, 17, 6, 14, 34, 3, 8, 10, 5, 0, 26, 2, 20, 11, 31, 25, 19, 13, 29, 15, 33]
(28, 480, 480, 3) (28, 15, 15, 3, 10)
18 (480, 480, 3) (15, 15, 3, 10)
(28, 15, 15, 3) and (15, 15, 3)


In [36]:
def binary_cross_entropy(logits, labels):
    epsilon = 1e-7
    logits = tf.clip_by_value(logits, epsilon, 1 - epsilon)
    return -(labels * tf.math.log(logits) +
             (1 - labels) * tf.math.log(1 - logits))

def my_sigmoid(x):
    result = 1/(1 + np.exp(-x))
    return result

def my_softmax(x):
    """Compute softmax values for each sets of scores in x."""
    result = np.exp(x) / np.sum(np.exp(x), axis=0)
    return result

In [37]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def xywh_to_x1y1x2y2(box):
    xy = box[..., 0:2]
    wh = box[..., 2:4]

    x1y1 = xy - wh / 2
    x2y2 = xy + wh / 2

    y_box = K.concatenate([x1y1, x2y2], axis=-1)
    return y_box

def broadcast_iou(box_a, box_b):
    """
    calculate iou between box_a and multiple box_b in a broadcast way
    inputs: box_a: a tensor full of boxes, eg. (B, N, 4), box is in x1y1x2y2
            box_b: another tensor full of boxes, eg. (B, M, 4)
    """

    # (B, N, 1, 4)
    box_a = tf.expand_dims(box_a, -2)
    # (B, 1, M, 4)
    box_b = tf.expand_dims(box_b, -3)
    # (B, N, M, 4)
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box_a), tf.shape(box_b))

    # (B, N, M, 4)
    # (B, N, M, 4)
    box_a = tf.broadcast_to(box_a, new_shape)
    box_b = tf.broadcast_to(box_b, new_shape)

    # (B, N, M, 1)
    al, at, ar, ab = tf.split(box_a, 4, -1)
    bl, bt, br, bb = tf.split(box_b, 4, -1)

    # (B, N, M, 1)
    left = tf.math.maximum(al, bl)
    right = tf.math.minimum(ar, br)
    top = tf.math.maximum(at, bt)
    bot = tf.math.minimum(ab, bb)

    # (B, N, M, 1)
    iw = tf.clip_by_value(right - left, 0, 1)
    ih = tf.clip_by_value(bot - top, 0, 1)
    i = iw * ih

    # (B, N, M, 1)
    area_a = (ar - al) * (ab - at)
    area_b = (br - bl) * (bb - bt)
    union = area_a + area_b - i

    # (B, N, M)
    iou = tf.squeeze(i / (union + 1e-7), axis=-1)

    return iou

## https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/yolov3.py#L213

def calc_ignore_mask(ignore_thresh, true_box, pred_box):
    
        # YOLOv3:
        # "If the bounding box prior is not the best but does overlap a ground
        # truth object by more than some threshold we ignore the prediction,
        # following [17]. We use the threshold of .5."
        # calculate the iou for each pair of pred bbox and true bbox, then find the best among them

        # (None, 13, 13, 3, 4)
        
        true_box_reorganised = xywh_to_x1y1x2y2(true_box)  # reorganised to x1, y1, x2, y2
        pred_box_reorganised = xywh_to_x1y1x2y2(pred_box)
        
        true_box_shape = tf.shape(true_box_reorganised)  
        # (None, 13, 13, 3, 4)
        pred_box_shape = tf.shape(pred_box_reorganised)  
        # (None, 507, 4)
        true_box_reorganised = tf.reshape(true_box_reorganised, [true_box_shape[0], -1, 4])
        # sort true_box to have non-zero boxes rank first
        true_box_reorganised = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING")
        # (None, 100, 4)
        # only use maximum 100 boxes per groundtruth to calcualte IOU, otherwise
        # GPU emory comsumption would explode for a matrix like (16, 52*52*3, 52*52*3, 4)
        true_box_reorganised = true_box_reorganised[:, 0:100, :]
        # (None, 507, 4)
        pred_box_reorganised = tf.reshape(pred_box_reorganised, [pred_box_shape[0], -1, 4])

        # https://github.com/dmlc/gluon-cv/blob/06bb7ec2044cdf3f433721be9362ab84b02c5a90/gluoncv/model_zoo/yolo/yolo_target.py#L198
        # (None, 507, 507)
        iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)
        # (None, 507)
        best_iou = tf.reduce_max(iou, axis=-1)
        # (None, 13, 13, 3)
        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)
        
        return ignore_mask

In [38]:
## made on 16/7/2020 at 8:48 pm

from functools import partial

anchors = anchors_wrt_target

def my_custom_loss(y_true, y_pred):
    
    def pre_loss(my_custom_loss, anchors):
        
        num_anchors = len(anchors)
        num_classes = 5
        ignore_thresh = 0.5
        grid_size = [15., 15.]
        grid_stride = 480. / grid_size[0]
        batch_shape = y_pred.get_shape()
#        batch_size = batch_shape[0]
        batch_size = 4.
    
        scaled_anchors = anchors / grid_stride
    
        Lambda_Coord = 5.0
        Lambda_no_obj = 0.5
    
        grid_x = np.arange(grid_size[1])
        grid_y = np.arange(grid_size[0])
    
        a = np.array(np.meshgrid(grid_x, grid_y))
        b = np.array(np.meshgrid(grid_x, grid_y))
        c = np.array(np.meshgrid(grid_x, grid_y))
        d = np.concatenate((a,b,c), axis = 0)
        e = d.transpose(2, 1, 0)
        grid_final = np.reshape(e,[1,15,15,3,2])
    
        tot_loss = tf.zeros(1, dtype='float32')

        obj_mask = y_true[..., 4:5]

## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

        pred_box_xy = K.sigmoid(y_pred[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
        pred_box_xy_wrt_target_image = (pred_box_xy * grid_stride) / 480.
        true_box_xy_wrt_target_image = y_true[..., :2]

        xy_arr = Lambda_Coord * K.square(true_box_xy_wrt_target_image - pred_box_xy_wrt_target_image)
    
        xy_loss = K.sum(xy_arr * obj_mask) / batch_size
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

        pred_box_wdht = K.exp(y_pred[..., 2:4]) * (anchors_wrt_target / 480.)
    
        true_box_wdht = y_true[..., 2:4]
    
        wh_arr = Lambda_Coord * K.square(true_box_wdht - pred_box_wdht)
    
        wh_loss = K.sum(wh_arr * obj_mask) / batch_size
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

        pred_obj_mask = K.sigmoid(y_pred[..., 4:5])  # shape = 28, 15, 15, 3, 1
       
        true_box_wrt_ti = K.concatenate([true_box_xy_wrt_target_image, true_box_wdht], axis = -1)  ## in x,y,w,h format
        pred_box_wrt_ti = K.concatenate([pred_box_xy_wrt_target_image, pred_box_wdht], axis = -1)  ## in x,y,w,h format
    
        ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
        
#        bce = tf.keras.losses.BinaryCrossentropy()        
#        obj_loss = K.sum(bce(obj_mask, pred_obj_mask) * obj_mask)

        obj_loss_arr = K.square(obj_mask - pred_obj_mask)
        obj_loss = K.sum(obj_loss_arr * obj_mask) / batch_size
    
        no_obj_mask = 1. - obj_mask
                
        noobj_loss_arr = Lambda_no_obj * K.square(obj_mask - pred_obj_mask)
        noobj_loss = K.sum(noobj_loss_arr * no_obj_mask * ignore_mask) / batch_size
        
        
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

        true_classes = y_true[..., 5:10]
    
        pred_classes = K.sigmoid(y_pred[..., 5:10])
        
#        cce = tf.keras.losses.CategoricalCrossentropy()
    
#        class_loss = K.sum(bce(true_classes, pred_classes) * obj_mask)

        class_loss_arr = K.square(true_classes - pred_classes)
        class_loss = K.sum(class_loss_arr * obj_mask) / batch_size

        tot_loss = xy_loss + wh_loss + obj_loss + noobj_loss + class_loss
        
        return tot_loss
    
    loss = pre_loss(my_custom_loss, anchors)
    
    return loss
    


In [39]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

my_model_1.compile(optimizer= opt, loss = my_custom_loss, metrics = ['accuracy'])

In [40]:
my_model_1.fit(X_train ,Y_train, epochs= 5, batch_size = 4, validation_data=(X_val,Y_val))


Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 709s 25s/step - loss: 134.9373 - accuracy: 0.0525 - val_loss: 95.8685 - val_accuracy: 0.0000e+00
Epoch 2/5
28/28 [==============================] - 164s 6s/step - loss: 104.5696 - accuracy: 0.0657 - val_loss: 92.2682 - val_accuracy: 0.1996
Epoch 3/5
28/28 [==============================] - 126s 5s/step - loss: 98.4767 - accuracy: 0.0887 - val_loss: 88.2615 - val_accuracy: 0.3057
Epoch 4/5
28/28 [==============================] - 126s 4s/step - loss: 91.2745 - accuracy: 0.1074 - val_loss: 83.0236 - val_accuracy: 0.3281
Epoch 5/5
28/28 [==============================] - 129s 5s/step - loss: 85.3401 - accuracy: 0.1209 - val_loss: 77.7573 - val_accuracy: 0.3296


In [41]:
my_model_1.fit(X_train ,Y_train, epochs= 100, batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/100
28/28 [==============================] - 119s 4s/step - loss: 77.1194 - accuracy: 0.1356 - val_loss: 73.0623 - val_accuracy: 0.3311
Epoch 2/100
28/28 [==============================] - 122s 4s/step - loss: 70.0128 - accuracy: 0.1565 - val_loss: 68.6755 - val_accuracy: 0.3311
Epoch 3/100
28/28 [==============================] - 127s 5s/step - loss: 66.0722 - accuracy: 0.1690 - val_loss: 64.2477 - val_accuracy: 0.3311
Epoch 4/100
28/28 [==============================] - 127s 5s/step - loss: 61.4282 - accuracy: 0.1778 - val_loss: 60.3022 - val_accuracy: 0.3311
Epoch 5/100
28/28 [==============================] - 130s 5s/step - loss: 58.2752 - accuracy: 0.1915 - val_loss: 56.7538 - val_accuracy: 0.3311
Epoch 6/100
28/28 [==============================] - 128s 5s/step - loss: 57.7774 - accuracy: 0.2032 - val_loss: 53.4900 - val_accuracy: 0.3311
Epoch 7/100
28/28 [==============================] - 134s 5s/step - loss: 52.1272 - accuracy:

In [42]:
my_model_1.fit(X_train ,Y_train, epochs= 300, batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/300
28/28 [==============================] - 118s 4s/step - loss: 12.9221 - accuracy: 0.3280 - val_loss: 22.2361 - val_accuracy: 0.3269
Epoch 2/300
28/28 [==============================] - 123s 4s/step - loss: 12.9571 - accuracy: 0.3263 - val_loss: 22.9135 - val_accuracy: 0.3259
Epoch 3/300
28/28 [==============================] - 127s 5s/step - loss: 12.7321 - accuracy: 0.3263 - val_loss: 21.4558 - val_accuracy: 0.3276
Epoch 4/300
28/28 [==============================] - 126s 5s/step - loss: 12.9877 - accuracy: 0.3252 - val_loss: 22.6777 - val_accuracy: 0.3278
Epoch 5/300
28/28 [==============================] - 127s 5s/step - loss: 12.5190 - accuracy: 0.3260 - val_loss: 24.5714 - val_accuracy: 0.3269
Epoch 6/300
28/28 [==============================] - 132s 5s/step - loss: 12.2564 - accuracy: 0.3256 - val_loss: 23.2792 - val_accuracy: 0.3270
Epoch 7/300
28/28 [==============================] - 128s 5s/step - loss: 12.3130 - accuracy:

28/28 [==============================] - 138s 5s/step - loss: 9.3882 - accuracy: 0.3450 - val_loss: 18.0394 - val_accuracy: 0.3569
Epoch 115/300
28/28 [==============================] - 127s 5s/step - loss: 9.4930 - accuracy: 0.3444 - val_loss: 17.6342 - val_accuracy: 0.3581
Epoch 116/300
28/28 [==============================] - 135s 5s/step - loss: 9.2826 - accuracy: 0.3464 - val_loss: 17.6007 - val_accuracy: 0.3587
Epoch 117/300
28/28 [==============================] - 130s 5s/step - loss: 9.6109 - accuracy: 0.3436 - val_loss: 18.2228 - val_accuracy: 0.3561
Epoch 118/300
28/28 [==============================] - 132s 5s/step - loss: 9.4065 - accuracy: 0.3442 - val_loss: 18.1563 - val_accuracy: 0.3539
Epoch 119/300
28/28 [==============================] - 128s 5s/step - loss: 9.2241 - accuracy: 0.3462 - val_loss: 18.1706 - val_accuracy: 0.3517
Epoch 120/300
28/28 [==============================] - 132s 5s/step - loss: 9.1377 - accuracy: 0.3452 - val_loss: 17.9657 - val_accuracy: 0.3502

Epoch 171/300
28/28 [==============================] - 130s 5s/step - loss: 8.7437 - accuracy: 0.3459 - val_loss: nan - val_accuracy: 0.3574
Epoch 172/300
28/28 [==============================] - 131s 5s/step - loss: 9.2333 - accuracy: 0.3471 - val_loss: nan - val_accuracy: 0.3602
Epoch 173/300
28/28 [==============================] - 130s 5s/step - loss: 8.6110 - accuracy: 0.3490 - val_loss: nan - val_accuracy: 0.3626
Epoch 174/300
28/28 [==============================] - 127s 5s/step - loss: 8.9125 - accuracy: 0.3485 - val_loss: nan - val_accuracy: 0.3604
Epoch 175/300
28/28 [==============================] - 130s 5s/step - loss: 8.6363 - accuracy: 0.3465 - val_loss: nan - val_accuracy: 0.3583
Epoch 176/300
28/28 [==============================] - 130s 5s/step - loss: 8.7295 - accuracy: 0.3467 - val_loss: nan - val_accuracy: 0.3578
Epoch 177/300
28/28 [==============================] - 132s 5s/step - loss: 8.8788 - accuracy: 0.3468 - val_loss: nan - val_accuracy: 0.3550
Epoch 178/300

28/28 [==============================] - 131s 5s/step - loss: 8.2088 - accuracy: 0.3515 - val_loss: nan - val_accuracy: 0.3633
Epoch 288/300
28/28 [==============================] - 133s 5s/step - loss: 8.1741 - accuracy: 0.3507 - val_loss: nan - val_accuracy: 0.3637
Epoch 289/300
28/28 [==============================] - 133s 5s/step - loss: 8.2049 - accuracy: 0.3493 - val_loss: nan - val_accuracy: 0.3633
Epoch 290/300
28/28 [==============================] - 133s 5s/step - loss: 8.0989 - accuracy: 0.3497 - val_loss: nan - val_accuracy: 0.3626
Epoch 291/300
28/28 [==============================] - 130s 5s/step - loss: 8.3958 - accuracy: 0.3525 - val_loss: nan - val_accuracy: 0.3646
Epoch 292/300
28/28 [==============================] - 133s 5s/step - loss: 8.2630 - accuracy: 0.3528 - val_loss: nan - val_accuracy: 0.3641
Epoch 293/300
28/28 [==============================] - 129s 5s/step - loss: 8.0950 - accuracy: 0.3500 - val_loss: nan - val_accuracy: 0.3637
Epoch 294/300
28/28 [======

In [46]:
xx = X_train[0]
xx = np.expand_dims(xx, axis = 0)

In [47]:
xx.shape

(1, 480, 480, 3)

In [48]:
pred_1 = my_model_1.predict(xx)
pred_1.shape

(1, 15, 15, 3, 10)

In [49]:
pred_1[0,3,4]

array([[ 2.6040742 , -2.6196136 , -0.29111266, -1.2238854 , -5.5518994 ,
        -0.3761594 ,  0.10476901,  0.93978536, -1.0036842 , -0.32729968],
       [-1.1468341 ,  0.17492291, -0.16110761,  1.1338296 , -3.6751573 ,
        -0.44184023, -0.10311824, -1.0567521 , -0.04028576, -3.1515012 ],
       [ 0.99488086, -1.2110277 , -0.50744426,  0.14199075,  3.7913659 ,
         4.180321  , -5.062862  , -6.4511056 , -4.4034314 , -2.9533648 ]],
      dtype=float32)

In [50]:
Y_train[0,3,4]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.296875  , 0.22945207, 0.32451922, 0.37671235, 1.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)

In [52]:
grid_size = [15., 15.]
grid_stride = 480. / grid_size[0]

grid_x = np.arange(grid_size[1])
grid_y = np.arange(grid_size[0])
    
a = np.array(np.meshgrid(grid_x, grid_y))
b = np.array(np.meshgrid(grid_x, grid_y))
c = np.array(np.meshgrid(grid_x, grid_y))
d = np.concatenate((a,b,c), axis = 0)
e = d.transpose(2, 1, 0)
grid_final = np.reshape(e,[1,15,15,3,2])


pred_1_xy = K.sigmoid(pred_1[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
pred_1_xy = (pred_1_xy * grid_stride) / 480.

print(K.eval(pred_1_xy[0,3,4]))

[[0.2620749  0.27119246]
 [0.21607119 0.30290797]
 [0.24867007 0.28196797]]


In [53]:
pred_1_wdht = K.exp(pred_1[..., 2:4]) * (anchors_wrt_target / 480.)
print(K.eval(pred_1_wdht[0,3,4]))

[[0.10600591 0.04001161]
 [0.22916937 0.2455336 ]
 [0.23806323 0.38506982]]


In [55]:
print(Y_train[0,1,9])
print(K.eval(pred_1_xy[0,1,9]))
print(K.eval(pred_1_wdht[0,1,9]))

[[0.66466343 0.09589041 0.17548077 0.15068492 1.         0.
  1.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]
[[0.13333161 0.6000078 ]
 [0.1091954  0.6081858 ]
 [0.06770397 0.66609347]]
[[0.17151742 0.12059849]
 [2.026833   0.08880548]
 [0.23238139 0.07307644]]


In [56]:
pred_1[0,1,9]

array([[10.562704  , -9.062217  ,  0.1900773 , -0.12058827,  3.9069479 ,
        -7.1260533 ,  2.6177287 , -3.4814658 , -4.0269628 , -5.471991  ],
       [ 0.5663959 , -1.9662929 ,  2.0186608 ,  0.11684419, -6.1889324 ,
        -5.2888823 , -2.0456536 ,  0.4903702 ,  2.218853  , -3.4079397 ],
       [-4.1473947 ,  4.7474995 , -0.53160065, -1.519928  , -6.7100854 ,
        -0.3332788 , -4.7860847 , -3.9409394 , -3.775373  , -0.01995534]],
      dtype=float32)

In [57]:
pred_1[0,1,12]

array([[ 8.8554649e+00, -8.3806791e+00,  1.4683717e-01, -5.9656873e-03,
         3.0014386e+00, -5.8467865e+00, -4.9454231e+00,  4.2439489e+00,
        -4.2812924e+00, -5.9560204e+00],
       [-4.4897345e-01,  8.4373468e-01,  1.0086120e+00, -1.6708611e-01,
        -6.6816216e+00, -2.6949046e+00, -2.1352654e+00, -2.7473166e+00,
         2.6926556e+00, -3.5140853e+00],
       [-6.2837296e+00,  5.1908946e+00, -1.1649001e-01, -1.2108866e+00,
        -7.2962475e+00, -9.7192240e-01, -2.5359476e+00, -2.5246689e+00,
        -3.5385990e+00,  2.2019839e-01]], dtype=float32)

In [58]:
print(Y_train[0,1,12])
print(K.eval(pred_1_xy[0,1,12]))
print(K.eval(pred_1_wdht[0,1,12]))

[[0.8605769  0.07876712 0.1201923  0.14383562 1.         0.
  0.         1.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]
[[0.13332383 0.8000153 ]
 [0.09264032 0.84661674]
 [0.06679086 0.8662976 ]]
[[0.16425903 0.13524519]
 [0.73817503 0.06685446]
 [0.3519505  0.09953878]]


In [59]:
yy = X_train[1]
yy = np.expand_dims(yy, axis = 0)
pred_2 = my_model_1.predict(yy)

pred_2_xy = K.sigmoid(pred_2[..., :2]) + grid_final
pred_2_xy = (pred_2_xy * grid_stride) / 480.

pred_2_wdht = K.exp(pred_2[..., 2:4]) * (anchors_wrt_target / 480.)

print(Y_train[1,2,2])
print(K.eval(pred_1_xy[0,2,2]))
print(K.eval(pred_1_wdht[0,2,2]))

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.19591346 0.13874345 0.38701922 0.2774869  1.         1.
  0.         0.         0.         0.        ]]
[[0.19723055 0.13394494]
 [0.14063706 0.1782419 ]
 [0.1519225  0.18421416]]
[[0.19325015 0.08368905]
 [0.13095404 0.10317025]
 [0.23133735 0.22518261]]


In [60]:
pred_2[0,2,2]

array([[ 3.2089632 , -3.6418161 ,  0.5830549 , -1.4382277 , -4.6716084 ,
        -1.8548412 , -0.38763475,  1.1856356 , -0.70323694, -1.1284423 ],
       [-1.1177062 , -0.61693   , -0.21265842,  1.1024228 , -3.8151162 ,
        -0.9173586 , -0.5837614 , -1.2895738 ,  0.04824073, -3.8286119 ],
       [-0.47356623, -0.0253053 , -0.13761845, -0.15889588,  2.82921   ,
         3.750211  , -5.3314214 , -7.264749  , -5.623835  , -3.9370337 ]],
      dtype=float32)

In [61]:
pred_2[0,5,9]

array([[ 7.4742904e+00, -7.6929173e+00,  4.6335384e-03,  3.0162108e-01,
         2.9726400e+00, -6.0349326e+00, -4.0324583e+00, -4.2022653e+00,
         3.3843787e+00, -7.0843639e+00],
       [ 7.6683432e-01, -6.0121268e-01,  7.3259830e-01,  3.7992400e-01,
        -6.8170829e+00, -4.0402102e+00, -3.7372422e+00, -1.7707355e+00,
         3.2379313e+00, -1.8671222e+00],
       [-4.7128291e+00,  4.6748919e+00, -5.8854735e-01, -1.9568634e+00,
        -8.6754513e+00, -1.0416988e+00, -3.4385278e+00, -4.0774121e+00,
        -4.6703849e+00,  5.4033136e-01]], dtype=float32)

In [62]:
print(Y_train[1,5,9])
print(K.eval(pred_1_xy[0,5,9]))
print(K.eval(pred_1_wdht[0,5,9]))

[[0.63461536 0.33769634 0.17788461 0.12041884 1.         0.
  0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]
[[0.39990443 0.60003954]
 [0.34325308 0.65145963]
 [0.33356088 0.6661117 ]]
[[0.25109175 0.09116796]
 [0.39423022 0.0844135 ]
 [0.08603773 0.05436709]]


In [63]:
zz = X_train[2]
zz = np.expand_dims(zz, axis = 0)
pred_3 = my_model_1.predict(zz)

pred_3_xy = K.sigmoid(pred_3[..., :2]) + grid_final
pred_3_xy = (pred_3_xy * grid_stride) / 480.

pred_3_wdht = K.exp(pred_3[..., 2:4]) * (anchors_wrt_target / 480.)

print(Y_train[2,2,5])
print('========================================')
print(pred_3[0,2,5])
print('========================================')
print(K.eval(pred_1_xy[0,2,5]))
print('========================================')
print(K.eval(pred_1_wdht[0,2,5]))

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.38942307 0.18201755 0.64903843 0.23245615 1.         1.
  0.         0.         0.         0.        ]]
[[ 1.6781175  -1.9051481   0.12965131 -1.3964186  -3.0297186  -0.81196404
   0.33130002  1.7986871  -0.6997856  -0.4304396 ]
 [ 0.9112919  -0.61311144  0.55573857  1.1611568  -3.07069     0.17931066
  -0.47110456 -0.77828854 -0.44484836 -1.8815792 ]
 [ 2.5751524  -3.068887    0.45351782 -0.53304267  2.7459445   3.5513444
  -3.0631416  -3.9864743  -3.7618346  -3.4708126 ]]
[[0.19914904 0.33403826]
 [0.16850671 0.3749523 ]
 [0.13971542 0.39535847]]
[[0.199841   0.09614696]
 [0.2441613  0.08432475]
 [0.35080305 0.36045757]]
